# Import Data and aggregatin in time


In [1]:
import sys
import pandas as pd
import numpy as np
%pylab inline
sys.version

Populating the interactive namespace from numpy and matplotlib


'2.7.13 |Intel Corporation| (default, Apr 27 2017, 15:33:46) \n[GCC 4.8.2 20140120 (Red Hat 4.8.2-15)]'

## Read data into dataframe

In [2]:
#df2=pd.read_csv('rain.txt.gz',delim_whitespace=True,parse_dates=True,index_col=[1,0],na_values=["-32767.0000000"])
#df2.sort_index(level=[0,1],inplace=True)
#df2.tz_localize('utc',level=1)[:4]
df=pd.read_pickle('pluvs.pkl.gz')

df=pd.read_csv('out_xpol.txt.gz',delim_whitespace=True,parse_dates=True,index_col=[1,0],na_values=["-32767.0000000"])
az=0.15
bz=0.39
akdp=40.6 #Ryzhkov and Zrnic (1996)
bkdp=0.866 #Ryzhkov and Zrnic (1996)
dtradar_x=2.0/60.0
df=df.assign(rz=(az/30)*((10**(df['Z_dbZ']/10))**bz)*dtradar_x)
df=df.assign(rkdp=(akdp/30)*(abs(df['Kdp_o_km'])**bkdp)*np.sign(df['Kdp_o_km'])*dtradar_x)

Filter rz for valid radar data (filtered by polatimetric variables)

df['rz'].where((df.Z_dbZ>=0) & (df.Z_dbZ<=50) & (df.Zdr_dbZr>=0) & (df.Zdr_dbZr<=3.5) & (df.Kdp_o_km>=0) & (df.Kdp_o_km<=5) 
               & (df.Rhv>0.85),inplace=True)
df['rkdp'].where((df.Z_dbZ>=0) & (df.Z_dbZ<=50) & (df.Zdr_dbZr>=0) & (df.Zdr_dbZr<=3.5) & (df.Kdp_o_km>=0) & (df.Kdp_o_km<=5) 
               & (df.Rhv>0.85),inplace=True)

In [3]:
#df=(df.merge(right=df2,left_index=True,right_index=True,how='outer',indicator='rain_merge'))
#df3=df.reset_index().merge(right=df2.reset_index(),how='inner',on=['estacao_codigo','datetime'],indicator=True)
#df3.set_index(['estacao_codigo','datetime'],inplace=True)
#pd.crosstab(df.rz>=-1,df.Rg_mm_10min>=-1)
#del df2
df3=pd.read_csv('out_spol.txt.gz',delim_whitespace=True,parse_dates=True,index_col=[1,0],na_values=["-32767.0000000"])
df=(df.merge(right=df3,left_index=True,right_index=True,how='outer',indicator='SPOL_merge'))
del df3
#df3[df3._merge=='both'].loc[5,:].plot()
#df3[df3._merge=='right_only'].loc[5,:]
dtradar_s=5.0/60.0
df['Rspol']=df.Rspol*dtradar_s

In [4]:
df.to_pickle("dataframe.pickle.gz",compression="gzip")

## Grouping data by station (using station code: 2nd columm of dataframe table)

In [5]:
# Group data by stations
station_grouped=df.reset_index(level=0).groupby('estacao_codigo')

## Let's resample the data only for R(Z-R) Rgauges and Raw gauges

In [6]:
station_2min=station_grouped.resample('2T',label='left').agg({'Rg_mm_10': sum, 'Rspol': sum})

station_10min=station_grouped.resample('10T',label='left').agg({'Rg_mm_10': sum, 'Rspol': sum})

station_1h=station_grouped.resample('1H',label='left').agg({'Rg_mm_10': sum, 'Rspol': sum})

station_1d=station_grouped.resample('1D',label='left').agg({'Rg_mm_10': sum, 'Rspol': sum})


In [7]:
station_1d.to_pickle('st1d.pkl.gz')

In [8]:
station_1h.to_pickle('st1h.pkl.gz')

In [9]:
station_10min.to_pickle('st10m.pkl.gz')

In [10]:
station_2min.to_pickle('st2m.pkl.gz')